In [19]:
import sys

sys.path.append("../")

In [20]:
import requests
import time
import pandas as pd
import numpy as np
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from stats_arb.tests import adf_test, kpss_test, cal_half_life, pp_test
from datetime import datetime, timedelta
from ta.volatility import BollingerBands

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
plt.ioff()

In [21]:
API_BASE = 'https://fapi.binance.com/fapi/v1/'
DATA_PATH = '/mnt/c/Users/vmodg/source/repos/BeanRepos/crypto-pair-trading/data/'
# DATA_PATH = '/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/crypto/'
TIMEFRAME = '1d'
lookback = 365

# TIMEFRAME = '1h'
# lookback = 30*4

# TIMEFRAME = '15m'
# lookback = 30*4


In [22]:


LABELS = [
    'open_time',
    'open',
    'high',
    'low',
    'close',
    'volume',
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]

DROP_COLUMNS=[
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]


def get_batch(symbol, interval='1m', start_time=0, limit=400):
    """Use a GET request to retrieve a batch of candlesticks. Process the JSON into a pandas
    dataframe and return it. If not successful, return an empty dataframe.
    """

    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'limit': limit
    }
    try:
        # timeout should also be given as a parameter to the function
        response = requests.get(f'{API_BASE}klines', params, timeout=30)
    except requests.exceptions.ConnectionError:
        print('Connection error, Cooling down for 5 mins...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    except requests.exceptions.Timeout:
        print('Timeout, Cooling down for 5 min...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    if response.status_code == 200:
        return pd.DataFrame(response.json(), columns=LABELS)
    
    print(f'Got erroneous response back {symbol}: {response}. {response.text}')
    return pd.DataFrame([])


def get_candles(base, quote, start_date: datetime, end_date=None, interval='1m'):
    batches = []

    if end_date is None:
        end_date = datetime.utcnow()

    last_timestamp = int(start_date.timestamp()) * 1000
    # gather all candlesticks available, starting from the last timestamp loaded from disk or 0
    # stop if the timestamp that comes back from the api is the same as the last one
    previous_timestamp = None

    while previous_timestamp != last_timestamp:
        # stop if we reached data from today
        if datetime.fromtimestamp(last_timestamp / 1000) >= end_date:
            break

        previous_timestamp = last_timestamp

        new_batch = get_batch(
            symbol=base + quote,
            interval=interval,
            start_time=last_timestamp
        )

        # requesting candles from the future returns empty
        # also stop in case response code was not 200
        if new_batch.empty:
            break

        last_timestamp = new_batch['open_time'].max()

        # sometimes no new trades took place yet on date.today();
        # in this case the batch is nothing new
        if previous_timestamp == last_timestamp:
            break

        batches.append(new_batch)
        last_datetime = datetime.fromtimestamp(last_timestamp / 1000)

        covering_spaces = 20 * ' '
        print(datetime.now(), base, quote, interval, str(last_datetime) + covering_spaces, end='\r', flush=True)

    if len(batches) > 0:
        # write clean version of csv to parquet
        df = pd.concat(batches, ignore_index=True)
        df.drop(columns=DROP_COLUMNS, inplace=True)
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        df.set_index('open_time', inplace=True)
        df = df[~df.index.duplicated(keep='first')]
        return df



In [23]:
# import requests

# url = "https://fapi.binance.com/fapi/v1/exchangeInfo"

# payload={}
# headers = {}

# response = requests.request("GET", url, headers=headers, data=payload).json()

# symbols = [s['symbol'] for s in response['symbols'] if s['contractType'] == 'PERPETUAL' and s['quoteAsset'] == 'USDT']
# # symbols[100:]
# symbols = symbols[:50]
# len(symbols)


In [24]:
# symbols = pd.read_csv(f'/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/symbols.csv')['symbol'].values.tolist()
symbols = ['BTCUSDT',
'ETHUSDT',
'XRPUSDT',
'LTCUSDT',
'ETCUSDT',
'LINKUSDT',
'XLMUSDT',
'ADAUSDT',
'BNBUSDT',
'ATOMUSDT',
'ALGOUSDT',
'DOTUSDT',
'SOLUSDT',
'AVAXUSDT',
'MATICUSDT',
'XMRUSDT',
'NEARUSDT',
'EOSUSDT',
'NEOUSDT',
'UNIUSDT']
symbols = [s.replace('USDT', '') for s in symbols]

data = []
start_time = datetime.utcnow() - timedelta(days=lookback)

for symbol in symbols:
    df = pd.read_csv(f'{DATA_PATH}{TIMEFRAME}/{symbol}-USDT.csv', parse_dates=['open_time'], index_col=['open_time'])
    df = df[df.index >= start_time].copy()
    # print('start time', df.index[-1].to_pydatetime())

    df1 = get_candles(
        base=symbol, 
        quote='USDT', 
        start_date=df.index[-1].to_pydatetime(),
        # end_date=datetime.now() - timedelta(days=15),
        interval=TIMEFRAME
    )
    if df1 is not None:
        df = pd.concat([df, df1])
        df = df[~df.index.duplicated(keep='first')]

    df.rename(columns={'close': symbol}, inplace=True)
    # the data is too long, just limit to recent period
    log = np.log(df[symbol].astype(np.float32))
    data.append(log)

df = pd.concat(data, axis=1)
df = df.dropna(axis=1, how='all')
# df.dropna(inplace=True, how='any')

# our of sample
# df = df[df.index <= datetime.now() - timedelta(days=15)].copy()

df.tail()

,BTC,ETH,XRP,LTC,ETC,LINK,XLM,ADA,BNB,ATOM,ALGO,DOT,SOL,AVAX,MATIC,XMR,NEAR,EOS,NEO,UNI
open_time,,,,,,,,,,,,,,,,,,,,
2023-05-20,10.206902,7.505817,-0.758433,4.525477,2.907720,1.871648,-2.422721,-1.005122,5.738828,2.361137,-1.805453,1.687510,3.008846,2.682322,-0.132161,5.012567,0.506215,-0.130109,2.241773,1.665440
2023-05-21,10.193646,7.497701,-0.782634,4.523092,2.885862,1.860197,-2.429056,-1.021374,5.724761,2.348228,-1.823250,1.662220,2.975785,2.660609,-0.155251,5.010169,0.474991,-0.161343,2.208714,1.631199
2023-05-22,10.197656,7.504760,-0.773490,4.508549,2.902740,1.876254,-2.429396,-1.000488,5.733891,2.349659,-1.825730,1.669027,2.971132,2.685873,-0.137539,5.020256,0.476234,-0.162519,2.287878,1.623735
2023-05-23,10.211336,7.524755,-0.765933,4.518086,2.910610,1.872110,-2.428488,-0.993712,5.747799,2.353278,-1.817693,1.686028,2.996932,2.688867,-0.117658,5.020850,0.496524,-0.150823,2.320032,1.623538
2023-05-24,10.193748,7.506784,-0.780231,4.477678,2.890983,1.847352,-2.436459,-1.013352,5.731852,2.337663,-1.843895,1.664873,2.969594,2.667298,-0.136393,5.008767,0.467500,-0.171975,2.332823,1.602212


In [25]:
p = 1
COINTEGRATION_CONFIDENCE_LEVEL = 90

# the 90%, 95%, and 99% confidence levels for the trace statistic and maximum 
# eigenvalue statistic are stored in the first, second, and third column of 
# cvt and cvm, respectively
confidence_level_cols = {
    90: 0,
    95: 1,
    99: 2
}
confidence_level_col = confidence_level_cols[COINTEGRATION_CONFIDENCE_LEVEL]


def test_johansen(symbol_pairs):
    df_t = df[symbol_pairs].copy()
    df_t.dropna(inplace=True)

    # The second and third parameters indicate constant term, with a lag of 1. 
    result = coint_johansen(df_t, 0, p)

    trace_crit_value = result.cvt[:, confidence_level_col]
    eigen_crit_value = result.cvm[:, confidence_level_col]
#     print("trace_crit_value",trace_crit_value)
#     print("eigen_crit_value",eigen_crit_value)
#     print("lr1",result.lr1)
#     print("lr2",result.lr2)

    # The trace statistic and maximum eigenvalue statistic are stored in lr1 and lr2;
    # see if they exceeded the confidence threshold
    if np.all(result.lr1 >= trace_crit_value) and np.all(result.lr2 >= eigen_crit_value):
        # print(f"{symbol_pairs} are cointegrated")
        # The first i.e. leftmost column of eigenvectors matrix, result.evec, contains the best weights.
        v1= result.evec[:,0:1]
        hr=v1/-v1[1] #to get the hedge ratio divide the best_eigenvector by the negative of the second component of best_eigenvector
        #the regression will be: close of symbList[1] = hr[0]*close of symbList[0] + error
        #where the beta of the regression is hr[0], also known as the hedge ratio, and
        #the error of the regression is the mean reverting residual signal that you need to predict, it is also known as the "spread"
        #the spread = close of symbList[1] - hr[0]*close of symbList[0] or alternatively (the same thing):
        #do a regression with close of symbList[0] as x and lose of symbList[1] as y, and take the residuals of the regression to be the spread.
        coint_pair = dict(hedge_ratio=v1[:, 0])
        for i, s in enumerate(symbol_pairs):
            coint_pair[f'sid_{i+1}'] = s

        cointegrating_pairs.append(coint_pair)



In [26]:
import traceback


def calculate_spread(df, coint_df, selected_row, hedge_ratio, nb_symbols=2):
    spread = None
    for i in range(nb_symbols):
        if spread is None:
            spread = df[coint_df[f'sid_{i + 1}'].iloc[selected_row]] * hedge_ratio[i]
        else:
            spread += df[coint_df[f'sid_{i + 1}'].iloc[selected_row]] * hedge_ratio[i]
    
    spread.dropna(inplace=True)
    return spread


critical_val = 0.02

def find_stationary_portfolio(coint_df):
    data = []
    for i, _ in coint_df.iterrows():
        try:
            hedge_ratio = coint_df.iloc[i]['hedge_ratio']
            _df = df.copy()
            spread = calculate_spread(_df, coint_df, i, hedge_ratio)
            p_val = adf_test(spread, verbose=False)
            if p_val < critical_val:
                half_life = cal_half_life(spread)
                pairs_name = coint_df[[col for col in coint_df.columns if 'sid' in col]].iloc[i].values
                # print(i, pairs_name, 'is stationary with half life', half_life)
                # print(' ')
                data.append({
                    'i': i,
                    'pairs': pairs_name,
                    'half_life': half_life
                })
        except:
            print(coint_df.iloc[i])
            # traceback.print_exc()
    
    return pd.DataFrame(data)


# find_stationary_portfolio(coint_df)

In [27]:

import itertools as it

nb_symbols = 2
cointegrating_pairs = []

#get symbol pairs
pairs = list(it.combinations(symbols, nb_symbols))

for pair in pairs:
    try:
        test_johansen(list(pair))
    except Exception:
        pass

coint_df = pd.DataFrame(cointegrating_pairs)
coint_df['i'] = coint_df.index
coint_df.head()

,hedge_ratio,sid_1,sid_2,i
0,"[1.1576119039553057, -11.306456315721881]",BTC,LINK,0
1,"[1.702791289124594, 5.090083152961605]",BTC,ATOM,1
2,"[1.8541347078324322, 6.190664658348528]",BTC,UNI,2
3,"[1.656330839849165, -11.939120596246973]",ETH,LINK,3
4,"[2.8554575801738156, -12.514430517206744]",ETH,XMR,4


In [28]:
coint_df.to_csv(f'coint_df_{TIMEFRAME}.csv')


# Result

In [29]:
import warnings

warnings.filterwarnings("ignore")

stationary_df = find_stationary_portfolio(coint_df)
stationary_df = pd.merge(stationary_df, coint_df, on='i')
stationary_df.dropna(inplace=True)
stationary_df['i'] = stationary_df.i.astype('int64')
stationary_df['half_life'] = stationary_df.half_life.astype('int64')
if len(stationary_df) > 0:
    stationary_df.sort_values(by=['half_life'], inplace=True)
stationary_df


,i,pairs,half_life,hedge_ratio,sid_1,sid_2
13,16,"[LINK, AVAX]",6,"[13.400195264949005, -2.621811102048339]",LINK,AVAX
18,21,"[LINK, NEO]",6,"[12.527296499738783, -2.3265818265895475]",LINK,NEO
17,20,"[LINK, EOS]",6,"[12.995621895045927, -3.6069621063236528]",LINK,EOS
16,19,"[LINK, NEAR]",6,"[12.046073280591784, -0.970905393194962]",LINK,NEAR
5,8,"[ETC, LINK]",6,"[2.432615674058532, -13.441651802279633]",ETC,LINK
7,10,"[LINK, XLM]",6,"[12.53391390833762, -2.885525916314503]",LINK,XLM
8,11,"[LINK, ADA]",6,"[12.4209087198491, -2.0581621124822718]",LINK,ADA
9,12,"[LINK, BNB]",6,"[12.275874143174843, -3.0745399375371862]",LINK,BNB
11,14,"[LINK, DOT]",6,"[12.63120253815419, -2.4670473563632367]",LINK,DOT
12,15,"[LINK, SOL]",6,"[11.971543035910084, -0.8890953028634105]",LINK,SOL


In [30]:
stationary_df.to_csv(f'stationary_df_{TIMEFRAME}.csv', index=False)
# stationary_df

In [31]:
ignored_symbols = ['AVAX', 'XMR', 'ETC', 'DOT']
# ignored_symbols= []
mask = ~(stationary_df['sid_1'].isin(ignored_symbols) | stationary_df['sid_2'].isin(ignored_symbols))
filtered_df = stationary_df[mask]
# filtered_df['hdiff'] = filtered_df['hedge_ratio'].apply(lambda x: np.sum(x))
# filtered_df.query('abs(hdiff) <= 10')
filtered_df

,i,pairs,half_life,hedge_ratio,sid_1,sid_2
18,21,"[LINK, NEO]",6,"[12.527296499738783, -2.3265818265895475]",LINK,NEO
17,20,"[LINK, EOS]",6,"[12.995621895045927, -3.6069621063236528]",LINK,EOS
16,19,"[LINK, NEAR]",6,"[12.046073280591784, -0.970905393194962]",LINK,NEAR
7,10,"[LINK, XLM]",6,"[12.53391390833762, -2.885525916314503]",LINK,XLM
8,11,"[LINK, ADA]",6,"[12.4209087198491, -2.0581621124822718]",LINK,ADA
9,12,"[LINK, BNB]",6,"[12.275874143174843, -3.0745399375371862]",LINK,BNB
12,15,"[LINK, SOL]",6,"[11.971543035910084, -0.8890953028634105]",LINK,SOL
19,22,"[LINK, UNI]",7,"[11.699694766355188, -1.4428249312133739]",LINK,UNI
14,17,"[LINK, MATIC]",7,"[10.551625281776824, 0.016298519489419322]",LINK,MATIC
0,0,"[BTC, LINK]",7,"[1.1576119039553057, -11.306456315721881]",BTC,LINK
